# Exploring patient 5 to create a first model

In [8]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from itertools import groupby
from statistics import mean
import random

window_length=20*256


In [3]:
d_5=pd.read_csv("patient_5.csv",index_col = [0])
d_5

,EEG Fp1-Ref,EEG Fp2-Ref,EEG F7-Ref,EEG F3-Ref,EEG Fz-Ref,EEG F4-Ref,EEG F8-Ref,EEG T3-Ref,EEG C3-Ref,EEG Cz-Ref,EEG C4-Ref,EEG T4-Ref,EEG T5-Ref,EEG P3-Ref,EEG Pz-Ref,EEG P4-Ref,EEG T6-Ref,EEG O1-Ref,EEG O2-Ref,is_seizure_target
0,0.353494,0.793573,1.000000,0.701917,0.393511,0.183450,0.846558,0.579165,0.018523,0.627815,0.863314,0.909277,0.289584,0.190606,0.209212,0.769056,0.731333,0.000000,0.179405,1
1,0.336246,0.862974,1.000000,0.638187,0.376305,0.179248,0.838919,0.609300,0.000000,0.605302,0.854773,0.920654,0.291398,0.199013,0.214853,0.768280,0.729059,0.017021,0.195751,1
2,0.268712,0.919813,1.000000,0.511203,0.356371,0.224427,0.850186,0.618169,0.000000,0.601201,0.863473,0.937578,0.305050,0.191920,0.219181,0.799121,0.757659,0.045152,0.243526,1
3,0.347118,0.805896,1.000000,0.618322,0.364362,0.234248,0.866393,0.616341,0.000000,0.607192,0.876478,0.956812,0.308363,0.211140,0.236517,0.800898,0.770108,0.051896,0.248591,1
4,0.425620,0.772917,1.000000,0.615228,0.351910,0.214835,0.839804,0.504690,0.000000,0.602062,0.856330,0.955345,0.270725,0.206211,0.235480,0.804234,0.765600,0.037186,0.232421,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
983291,0.358651,0.000000,0.446580,0.350410,0.431132,0.438771,0.431005,0.601553,0.432301,0.466851,0.486067,1.000000,0.440335,0.429410,0.481188,0.497280,0.360721,0.479454,0.489994,0
983292,0.027400,0.011211,0.029130,0.000000,0.017762,0.040012,0.040446,0.011096,0.022597,0.031731,0.056863,0.153933,0.036740,0.039975,0.053607,0.071216,0.108301,0.072665,0.062631,0
983293,0.040683,0.080363,0.029821,0.027907,0.026126,0.036374,0.037631,0.000000,0.028419,0.028432,0.038594,0.019143,0.034508,0.037605,0.037596,0.044365,0.081376,0.047479,0.040991,0
983294,0.966625,1.000000,0.959231,0.976907,0.964426,0.956003,0.956450,0.953622,0.962633,0.956752,0.946422,0.872591,0.957035,0.956739,0.947944,0.940503,0.937985,0.941369,0.944088,0


# Window decision

### Functions

In [4]:
sampling_rate=256
def time_of_seizure(df,column):
    list_of_time=[]
    start_seizure=[]
    end_seizure=[]
    start_seizure_seconds=[]
    end_seizure_seconds=[]
    if df[column][0]==1:
        list_of_time.append(0)
    for i in range(len(df)-1):    
        if df[column][i]!=df[column][i+1]:
            list_of_time.append(i+1)
    for j in range(len(list_of_time)):
        if j%2==0:
            start_seizure.append(list_of_time[j])
            start_seizure_seconds.append(list_of_time[j]//sampling_rate)
        else:
            end_seizure.append(list_of_time[j])
            end_seizure_seconds.append(list_of_time[j]//sampling_rate)
  
    return start_seizure,start_seizure_seconds, end_seizure, end_seizure_seconds

def duration_each_seizure(df,column):
    is_seizure_=list(df[column])
   
    dur_list_=[]
    
    list_=[sum(g) for i, g in groupby(is_seizure_) if i == 1]
   
    for elem in list_:
        dur_list_.append(elem/256)
 
    return dur_list_
    
def number_total_seizures(df,column):
    is_seizure_=list(df[column])
   
    list_=[sum(g) for i, g in groupby(is_seizure_) if i == 1]
  
    return len(list_)

In [5]:
time_of_seizure(d_5,"is_seizure_target")

([0, 66304, 253952, 429824, 680448],
 [0, 259, 992, 1679, 2658],
 [32512, 225280, 370944, 641536, 982784],
 [127, 880, 1449, 2506, 3839])

#### Seizure windows on patient 5 

In [54]:
d_5_i=d_5.copy()
d_5_i_b=d_5_i[0:0+window_length]
d_5_ii_m=d_5_i[70000:70000+window_length]
d_5_iii_e=d_5_i[370943-window_length:370943]

#### Healthy windows on patient 5

In [55]:
d_5_h_b=d_5_i[32512:32512+window_length]
d_5_h_m=d_5_i[400000:400000+window_length]
d_5_h_e=d_5_i[680447-window_length:680447]

#### Exporting windows to csvs

In [56]:
d_5_i_b.to_csv('5_firstwindow_seizure_b.csv')
d_5_ii_m.to_csv("5_secondwindow_seizure_m.csv")
d_5_iii_e.to_csv("5_thirdwindow_seizure_e.csv")
d_5_h_b.to_csv("5_firswindow_healthy_b.csv")
d_5_h_m.to_csv("5_secondwindow_healthy_m.csv")
d_5_h_e.to_csv("5_thirdwindow_healthy_e.csv")

## Generalized window creation

In [11]:
# Function creating seizure windows
def create_seizure_windows(df):
    list_s=time_of_seizure(df,"is_seizure_target")[0]
    list_h=time_of_seizure(df,"is_seizure_target")[2]
    b=list_s[0]
    d_f=df.copy()
    d_s_b=d_f[b:b+window_length]
    return d_s_b
    
    

In [12]:
# Function creating healthy windows
def create_healthy_windows(df):
    list_s=time_of_seizure(df,"is_seizure_target")[0]
    list_h=time_of_seizure(df,"is_seizure_target")[2]
    b=list_h[0]
    d_f=df.copy()
    d_h_b=d_f[b:b+window_length]
    return d_h_b

In [13]:
d1_h=create_healthy_windows(d_5)

In [14]:
#Function creating random windows

def create_random_windows(df):
    b = random.randint(0,len(df))
    m= random.randint(0,len(df))
    e= random.randint(0,len(df))
    
    d_f=df.copy()
    d_t_b=d_f[b:b+window_length]
    d_t_m=d_f[m:m+window_length]
    d_t_e=d_f[e-window_length:e]
    
    return d_t_b,d_t_m,d_t_e


In [31]:
def split_dataframe(df):
    df_=df.copy()
    a=len(df)
    b=(a*30)//100
    df_train=df_[0:b]
    df_test=df_[b:a-1]
    df_test.reset_index(inplace=True)
    df_test.drop(columns="index",inplace=True)

    return df_train,df_test
    
    

In [32]:
df_train,df_test=split_dataframe(d_5)

/tmp/ipykernel_16883/2625881544.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.drop(columns="index",inplace=True)


In [40]:
d5_h=create_healthy_windows(df_train)
d5_s=create_seizure_windows(df_train)
d5_r1,d5_r2,d5_r3=create_random_windows(df_train)
d5_t1,d5_t2,d5_t3=create_random_windows(df_test)

In [41]:
d5_h.to_csv("5_train_healthy.csv")
d5_s.to_csv("5_train_seiwure.csv")
d5_r1.to_csv("5_train_random_1.csv")
d5_r2.to_csv("5_train_random_2.csv")
d5_r3.to_csv("5_train_random_3.csv")
d5_t1.to_csv("5_test_random_1.csv")
d5_t2.to_csv("5_test_random_2.csv")
d5_t3.to_csv("5_test_random_3.csv")